In [ ]:
# Part 1: Data Loading and Preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Load Control and Test Datasets with Correct Delimiter
control_file = '/content/control_group.csv'
test_file = '/content/test_group.csv'

try:
    control_data = pd.read_csv(control_file, sep=';')
    print(f"Control group dataset '{control_file}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file '{control_file}' was not found. Please check the file path.")

try:
    test_data = pd.read_csv(test_file, sep=';')
    print(f"Test group dataset '{test_file}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file '{test_file}' was not found. Please check the file path.")

# 3. Add Group Identifier Column
control_data['Group'] = 'Control'
test_data['Group'] = 'Test'

# 4. Combine the Datasets
combined_data = pd.concat([control_data, test_data], axis=0).reset_index(drop=True)

print(f"\nCombined dataset shape: {combined_data.shape}")
print(f"Number of Control group records: {(combined_data['Group'] == 'Control').sum()}")
print(f"Number of Test group records: {(combined_data['Group'] == 'Test').sum()}")

# 5. Inspect the Combined Data
print("\nFirst 5 rows of the combined dataset:")
print(combined_data.head())

print("\nDataset Information:")
print(combined_data.info())

print("\nStatistical Summary:")
print(combined_data.describe())

print("\nChecking for missing values:")
print(combined_data.isnull().sum())

# 6. Handle Missing Values
missing_values = combined_data.isnull().sum()
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    print("\nColumns with missing values:")
    print(missing_values)

    # Strategies used to handle missing values:
    # - For numerical columns: fill with mean or median
    # - For categorical columns: fill with mode or a placeholder

    numerical_cols = combined_data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numerical_cols:
        if combined_data[col].isnull().sum() > 0:
            median = combined_data[col].median()
            combined_data[col].fillna(median, inplace=True)
            print(f"Filled missing values in '{col}' with median value {median}.")

    categorical_cols = combined_data.select_dtypes(include=['object']).columns.tolist()
    for col in categorical_cols:
        if combined_data[col].isnull().sum() > 0:
            mode = combined_data[col].mode()[0]
            combined_data[col].fillna(mode, inplace=True)
            print(f"Filled missing values in '{col}' with mode value '{mode}'.")
else:
    print("\nNo missing values detected.")

# Verify that there are no more missing values
print("\nMissing values after imputation:")
print(combined_data.isnull().sum())

# 7. Convert 'Date' column to datetime
if 'Date' in combined_data.columns:
    combined_data['Date'] = pd.to_datetime(combined_data['Date'], format='%d.%m.%Y', errors='coerce')
    if combined_data['Date'].isnull().sum() > 0:
        print("\nThere were errors converting some 'Date' entries to datetime. Please check the data.")
    else:
        print("\n'Date' column successfully converted to datetime.")
else:
    print("\n'Date' column not found in the dataset.")

print("\nData types after conversion:")
print(combined_data.dtypes)

# 8. Create Additional Features like Calculate Click-Through Rate (CTR) and Conversion Rate
# CTR (%) = (# of Website Clicks / # of Impressions) * 100
# Conversion Rate (%) = (# of Purchase / # of Website Clicks) * 100

def calculate_ctr(row):
    try:
        return (row['# of Website Clicks'] / row['# of Impressions']) * 100
    except ZeroDivisionError:
        return 0

def calculate_conversion_rate(row):
    try:
        return (row['# of Purchase'] / row['# of Website Clicks']) * 100
    except ZeroDivisionError:
        return 0

combined_data['CTR (%)'] = combined_data.apply(calculate_ctr, axis=1)
combined_data['Conversion Rate (%)'] = combined_data.apply(calculate_conversion_rate, axis=1)

print("\nAdded 'CTR (%)' and 'Conversion Rate (%)' columns.")

# Display the first 5 rows of the cleaned and combined data
print("\nFirst 5 rows of the cleaned and combined dataset:")
print(combined_data.head())


Control group dataset '/content/control_group.csv' loaded successfully.
Test group dataset '/content/test_group.csv' loaded successfully.

Added 'Group' column to both datasets.

Combined dataset shape: (60, 11)
Number of Control group records: 30
Number of Test group records: 30

First 5 rows of the combined dataset:
      Campaign Name       Date  Spend [USD]  # of Impressions     Reach  \
0  Control Campaign  1.08.2019         2280           82702.0   56930.0   
1  Control Campaign  2.08.2019         1757          121040.0  102513.0   
2  Control Campaign  3.08.2019         2343          131711.0  110862.0   
3  Control Campaign  4.08.2019         1940           72878.0   61235.0   
4  Control Campaign  5.08.2019         1835               NaN       NaN   

   # of Website Clicks  # of Searches  # of View Content  # of Add to Cart  \
0               7016.0         2290.0             2159.0            1819.0   
1               8110.0         2033.0             1841.0            1219.

In [ ]:
!pip install scipy statsmodels


In [ ]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests


In [ ]:
def check_normality(data, column, group_col='Group'):
    """
    Performs Shapiro-Wilk Test for normality on a specified column for each group.

    Returns:
    - Dictionary with group names as keys and p-values as values.
    """
    groups = data[group_col].unique()
    normality_results = {}

    for group in groups:
        group_data = data[data[group_col] == group][column].dropna()
        stat, p = stats.shapiro(group_data)
        normality_results[group] = p
        print(f"Shapiro-Wilk Test for {column} in {group} group: p-value = {p:.4f}")

    return normality_results


In [ ]:
def check_homoscedasticity(data, column, group_col='Group'):
    """
    Performs Levene’s Test for homogeneity of variances.

    Returns:
    - Levene’s Test statistic and p-value.
    """
    groups = data[group_col].unique()
    group_data = [data[data[group_col] == group][column].dropna() for group in groups]
    stat, p = stats.levene(*group_data)
    print(f"Levene’s Test for {column}: statistic = {stat:.4f}, p-value = {p:.4f}")
    return stat, p


In [ ]:
def perform_mannwhitneyu(data, column, group_col='Group', alternative='two-sided'):
    """
    Performs Mann-Whitney U Test between two groups for a specified column.

    Returns:
    - U statistic and p-value.
    """
    group1 = data[data[group_col] == 'Control'][column].dropna()
    group2 = data[data[group_col] == 'Test'][column].dropna()
    stat, p = stats.mannwhitneyu(group1, group2, alternative=alternative)
    print(f"Mann-Whitney U Test for {column}: U-statistic = {stat}, p-value = {p:.4f}")
    return stat, p


In [ ]:
def perform_ttest(data, column, group_col='Group', equal_var=True, alternative='two-sided'):
    """
    Performs Independent Two-Sample t-Test between two groups for a specified column.

    Parameters:
    - equal_var: If False, perform Welch’s t-Test.

    Returns:
    - t statistic and p-value.
    """
    group1 = data[data[group_col] == 'Control'][column].dropna()
    group2 = data[data[group_col] == 'Test'][column].dropna()
    stat, p = stats.ttest_ind(group1, group2, equal_var=equal_var, alternative=alternative)
    test_type = "Independent Two-Sample t-Test" if equal_var else "Welch's t-Test"
    print(f"{test_type} for {column}: t-statistic = {stat:.4f}, p-value = {p:.4f}")
    return stat, p


In [ ]:
def test_metric(data, column, group_col='Group', alpha=0.05):
    """
    Tests a single metric by checking assumptions and performing the appropriate test.

    Returns:
    - Dictionary with test results.
    """
    print(f"\n--- Testing {column} ---")
    results = {}

    # Check Normality
    normality = check_normality(data, column, group_col)
    results['Normality'] = normality

    # Check Homoscedasticity
    homoscedasticity_stat, homoscedasticity_p = check_homoscedasticity(data, column, group_col)
    results['Homoscedasticity'] = {'Statistic': homoscedasticity_stat, 'p-value': homoscedasticity_p}

    # Decide on Test
    # If both groups are normal and variances are equal, use t-Test. Else, use Mann-Whitney U Test
    groups_normal = all(p > alpha for p in normality.values())
    homoscedastic = homoscedasticity_p > alpha

    if groups_normal and homoscedastic:
        print("Assumptions met for Independent Two-Sample t-Test.")
        stat, p = perform_ttest(data, column, group_col, equal_var=True)
        results['Test'] = {'Test Type': 'Independent Two-Sample t-Test', 'Statistic': stat, 'p-value': p}
    else:
        print("Assumptions not met. Proceeding with Mann-Whitney U Test.")
        stat, p = perform_mannwhitneyu(data, column, group_col)
        results['Test'] = {'Test Type': 'Mann-Whitney U Test', 'Statistic': stat, 'p-value': p}

    return results


In [ ]:
# List of metrics to test
metrics_to_test = [
    'Conversion Rate (%)',
    'CTR (%)',
    'Spend [USD]',
    '# of Website Clicks',
    '# of View Content',
    '# of Add to Cart',
    '# of Purchase'
]

# Dictionary to store test results
test_results = {}

for metric in metrics_to_test:
    result = test_metric(combined_data, metric, group_col='Group', alpha=0.05)
    test_results[metric] = result



--- Testing Conversion Rate (%) ---
Shapiro-Wilk Test for Conversion Rate (%) in Control group: p-value = 0.0050
Shapiro-Wilk Test for Conversion Rate (%) in Test group: p-value = 0.0373
Levene’s Test for Conversion Rate (%): statistic = 2.9044, p-value = 0.0937
Assumptions not met. Proceeding with Mann-Whitney U Test.
Mann-Whitney U Test for Conversion Rate (%): U-statistic = 520.0, p-value = 0.3042

--- Testing CTR (%) ---
Shapiro-Wilk Test for CTR (%) in Control group: p-value = 0.3438
Shapiro-Wilk Test for CTR (%) in Test group: p-value = 0.0004
Levene’s Test for CTR (%): statistic = 10.1817, p-value = 0.0023
Assumptions not met. Proceeding with Mann-Whitney U Test.
Mann-Whitney U Test for CTR (%): U-statistic = 203.0, p-value = 0.0003

--- Testing Spend [USD] ---
Shapiro-Wilk Test for Spend [USD] in Control group: p-value = 0.2313
Shapiro-Wilk Test for Spend [USD] in Test group: p-value = 0.1171
Levene’s Test for Spend [USD]: statistic = 0.0658, p-value = 0.7984
Assumptions met f

In [ ]:
# Collect p-values
p_values = [test_results[metric]['Test']['p-value'] for metric in metrics_to_test]

# Apply Bonferroni Correction
reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')

# Add corrected p-values and rejection status to test_results
for i, metric in enumerate(metrics_to_test):
    test_results[metric]['Test']['p-value_corrected'] = pvals_corrected[i]
    test_results[metric]['Test']['Reject_H0'] = reject[i]

# Display the results in a DataFrame
results_df = pd.DataFrame({
    'Metric': metrics_to_test,
    'Test Type': [test_results[metric]['Test']['Test Type'] for metric in metrics_to_test],
    'Statistic': [test_results[metric]['Test']['Statistic'] for metric in metrics_to_test],
    'p-value': [test_results[metric]['Test']['p-value'] for metric in metrics_to_test],
    'p-value_corrected': [test_results[metric]['Test']['p-value_corrected'] for metric in metrics_to_test],
    'Reject H0': [test_results[metric]['Test']['Reject_H0'] for metric in metrics_to_test]
})

print("\n--- Statistical Test Results with Bonferroni Correction ---")
display(results_df)



--- Statistical Test Results with Bonferroni Correction ---


,Metric,Test Type,Statistic,p-value,p-value_corrected,Reject H0
0,Conversion Rate (%),Mann-Whitney U Test,520.000000,0.304177,1.000000,False
1,CTR (%),Mann-Whitney U Test,203.000000,0.000268,0.001876,True
2,Spend [USD],Independent Two-Sample t-Test,-2.969992,0.004326,0.030284,True
3,# of Website Clicks,Mann-Whitney U Test,350.000000,0.141272,0.988904,False
4,# of View Content,Independent Two-Sample t-Test,0.490301,0.625770,1.000000,False
5,# of Add to Cart,Mann-Whitney U Test,684.500000,0.000541,0.003787,True
6,# of Purchase,Mann-Whitney U Test,454.000000,0.958727,1.000000,False


In [ ]:
def rank_biserial_correlation(data, column, group_col='Group'):
    """
    Calculates rank-biserial correlation for Mann-Whitney U Test.

    Returns:
    - Rank-biserial correlation coefficient.
    """
    group1 = data[data[group_col] == 'Control'][column].dropna()
    group2 = data[data[group_col] == 'Test'][column].dropna()
    U, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')
    n1 = len(group1)
    n2 = len(group2)
    rbc = 1 - (2 * U) / (n1 * n2)
    return rbc

# Example Usage:
metric = 'Conversion Rate (%)'
rbc = rank_biserial_correlation(combined_data, metric, group_col='Group')
print(f"Rank-Biserial Correlation for {metric}: {rbc:.4f}")


Rank-Biserial Correlation for Conversion Rate (%): -0.1556


In [ ]:
def cohen_d(group1, group2):
    """
    Calculates Cohen's d for two independent samples.

    Returns:
    - Cohen's d
    """
    mean1, mean2 = np.mean(group1), np.mean(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((var1 + var2) / 2))
    d = (mean1 - mean2) / pooled_std
    return d

# Example Usage:
metric = 'Spend [USD]'
group1 = combined_data[combined_data['Group'] == 'Control'][metric].dropna()
group2 = combined_data[combined_data['Group'] == 'Test'][metric].dropna()
d = cohen_d(group1, group2)
print(f"Cohen's d for {metric}: {d:.4f}")


Cohen's d for Spend [USD]: -0.7668


In [ ]:
# Define the test results
test_results = pd.DataFrame({
    'Metric': [
        'Conversion Rate (%)',
        'CTR (%)',
        'Spend [USD]',
        '# of Website Clicks',
        '# of View Content',
        '# of Add to Cart',
        '# of Purchase'
    ],
    'Test Type': [
        'Mann-Whitney U Test',
        'Mann-Whitney U Test',
        'Independent Two-Sample t-Test',
        'Mann-Whitney U Test',
        'Independent Two-Sample t-Test',
        'Mann-Whitney U Test',
        'Mann-Whitney U Test'
    ],
    'Statistic': [
        520.0,
        203.0,
        -2.969992,
        350.0,
        0.490301,
        684.5,
        454.0
    ],
    'p-value': [
        0.304177,
        0.000268,
        0.004326,
        0.141272,
        0.625770,
        0.000541,
        0.958727
    ],
    'p-value_corrected': [
        1.000000,
        0.001876,
        0.030284,
        0.988904,
        1.000000,
        0.003787,
        1.000000
    ],
    'Reject H0': [
        False,
        True,
        True,
        False,
        False,
        True,
        False
    ]
})


In [ ]:
# Initialize a list to store effect sizes
effect_sizes = []

# Iterate through each metric and compute effect size based on test type
for idx, row in test_results.iterrows():
    metric = row['Metric']
    test_type = row['Test Type']

    if test_type == 'Mann-Whitney U Test':
        # Compute Rank-Biserial Correlation
        rbc = rank_biserial_correlation(combined_data, metric, group_col='Group')
        effect_sizes.append(rbc)
    elif test_type == 'Independent Two-Sample t-Test':
        # Compute Cohen's d
        group1 = combined_data[combined_data['Group'] == 'Control'][metric].dropna()
        group2 = combined_data[combined_data['Group'] == 'Test'][metric].dropna()
        d = cohen_d(group1, group2)
        effect_sizes.append(d)
    else:
        # If other test types exist, handle accordingly
        effect_sizes.append(np.nan)

# Add the effect sizes to the test_results DataFrame
test_results['Effect Size'] = effect_sizes


In [ ]:
print("--- Statistical Test Results with Bonferroni Correction and Effect Sizes ---")
display(test_results)


--- Statistical Test Results with Bonferroni Correction and Effect Sizes ---


,Metric,Test Type,Statistic,p-value,p-value_corrected,Reject H0,Effect Size
0,Conversion Rate (%),Mann-Whitney U Test,520.000000,0.304177,1.000000,False,-0.155556
1,CTR (%),Mann-Whitney U Test,203.000000,0.000268,0.001876,True,0.548889
2,Spend [USD],Independent Two-Sample t-Test,-2.969992,0.004326,0.030284,True,-0.766849
3,# of Website Clicks,Mann-Whitney U Test,350.000000,0.141272,0.988904,False,0.222222
4,# of View Content,Independent Two-Sample t-Test,0.490301,0.625770,1.000000,False,0.126595
5,# of Add to Cart,Mann-Whitney U Test,684.500000,0.000541,0.003787,True,-0.521111
6,# of Purchase,Mann-Whitney U Test,454.000000,0.958727,1.000000,False,-0.008889
